## Data Quality Check Using Python




In [85]:
import pandas as pd
import numpy as np
from datetime import datetime

In [61]:
def read_file(file):
    file_name= pd.read_csv(file)
    return file_name

claims = read_file('sample_claims.csv')
claims.head(5)

,claim_id,patient_id,diagnosis_codes,procedure_code,date_service,date_received
0,1.0,A1670,Z01.419^Z11.51,99999,1/25/2021,1/26/2021
1,2.0,A0086,Z01.419^Z12.4,99999,1/27/2021,1/29/2021
2,3.0,A0086,Z12.4^Z11.51,87491,1/7/2021,1/10/2021
3,4.0,A0086,R30.0,86735,1/15/2021,1/16/2021
4,5.0,A0311,Z34.81^I10^G40.909^E66.9,83014,1/6/2021,1/7/2021


In [62]:
claims.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   claim_id         4998 non-null   float64
 1   patient_id       4994 non-null   object 
 2   diagnosis_codes  3714 non-null   object 
 3   procedure_code   4997 non-null   object 
 4   date_service     4990 non-null   object 
 5   date_received    5000 non-null   object 
dtypes: float64(1), object(5)
memory usage: 234.5+ KB


In [63]:
valid_procedure = read_file('valid_cpt_codes.csv')
valid_procedure.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2861 entries, 0 to 2860
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   code               2861 non-null   int64 
 1   short_description  2861 non-null   object
dtypes: int64(1), object(1)
memory usage: 44.8+ KB


In [64]:
#lets convert the code from valid procedure table data type into string so that it would match with table, claims
valid_procedure['code']= valid_procedure['code'].astype(str)
valid_procedure['procedure_code']= valid_procedure['code']
#valid_procedure['procedure_code']
valid_procedure.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2861 entries, 0 to 2860
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   code               2861 non-null   object
 1   short_description  2861 non-null   object
 2   procedure_code     2861 non-null   object
dtypes: object(3)
memory usage: 67.2+ KB


In [65]:
# join table valid_procedure to table claims (right join), it would give us the claims with valid procedure code

valid_code_claim = claims.merge(valid_procedure, on='procedure_code', how='inner')
#valid_code_claim = valid_code_claim[valid_code_claim['claim_id'].notna() | valid_code_claim[valid_code_claim].notna()]
valid_code_claim.head()

,claim_id,patient_id,diagnosis_codes,procedure_code,date_service,date_received,code,short_description
0,3.0,A0086,Z12.4^Z11.51,87491,1/7/2021,1/10/2021,87491,CHYLMD TRACH DNA AMP PROBE
1,22.0,A0311,NaN,87491,1/6/2021,1/7/2021,87491,CHYLMD TRACH DNA AMP PROBE
2,131.0,A0482,Z34.81^Z32.01^Z11.3^Z68.41,87491,1/27/2021,1/28/2021,87491,CHYLMD TRACH DNA AMP PROBE
3,132.0,A0482,N76.0^Z11.3,87491,1/14/2021,1/15/2021,87491,CHYLMD TRACH DNA AMP PROBE
4,146.0,A1019,NaN,87491,1/7/2021,1/8/2021,87491,CHYLMD TRACH DNA AMP PROBE


In [66]:
#valid_code_claim.to_csv('check.csv',index=False)

### Q1. a Top 5 common valid procedure codes

In [67]:
valid_code_claim['code'].value_counts().sort_values(ascending=False).head(5)

88175    155
87591    123
87491    122
87798    116
85049    107
Name: code, dtype: int64

### Q1.b Number of patients associated with top 5 valid procedure codes

In [68]:
pv_table = valid_code_claim.pivot_table(index=['code'], values=['patient_id', 'procedure_code'], 
                aggfunc={'procedure_code': 'count', 'patient_id': lambda x: len(x.unique())})
                                         #.sort_values(by=['procedure_code'],ascending=False))
pv_table.sort_values(by=['procedure_code'], ascending=False).head(5)

,patient_id,procedure_code
code,,
88175,38,155
87591,31,123
87491,37,122
87798,31,116
85049,32,107


### Q2. Top 5 most common valid diagnosis codes

In [69]:
icd_breakdown = claims.assign(diagnosis_codes=claims['diagnosis_codes'].str.split('^')).explode('diagnosis_codes')
icd_breakdown.head(5)

,claim_id,patient_id,diagnosis_codes,procedure_code,date_service,date_received
0,1.0,A1670,Z01.419,99999,1/25/2021,1/26/2021
0,1.0,A1670,Z11.51,99999,1/25/2021,1/26/2021
1,2.0,A0086,Z01.419,99999,1/27/2021,1/29/2021
1,2.0,A0086,Z12.4,99999,1/27/2021,1/29/2021
2,3.0,A0086,Z12.4,87491,1/7/2021,1/10/2021


In [70]:
icd_breakdown['code']= icd_breakdown['diagnosis_codes'].apply(lambda x: str(x).split('.')[0] )
icd_breakdown.head()

,claim_id,patient_id,diagnosis_codes,procedure_code,date_service,date_received,code
0,1.0,A1670,Z01.419,99999,1/25/2021,1/26/2021,Z01
0,1.0,A1670,Z11.51,99999,1/25/2021,1/26/2021,Z11
1,2.0,A0086,Z01.419,99999,1/27/2021,1/29/2021,Z01
1,2.0,A0086,Z12.4,99999,1/27/2021,1/29/2021,Z12
2,3.0,A0086,Z12.4,87491,1/7/2021,1/10/2021,Z12


In [71]:
# validate whether the icd_breakdown dataframe code column contains punctuations
pd.set_option('display.max_rows', 131)
icd_breakdown['code'].value_counts()

Z11    1474
nan    1286
Z34     861
Z01     529
E55     498
Z12     427
E78     337
N92     332
Z00     296
Z32     251
Z13     237
N76     226
E03     216
R53     210
R73     160
Z79     158
N95     158
Z31     154
N89     144
O09     138
I10     126
Z83     112
M25     110
R63     104
N93      98
Z30      85
G47      77
M79      74
F32      72
B18      67
Z68      67
F33      65
N91      65
R10      62
J30      56
R05      55
D64      53
Z20      53
F41      52
R19      51
M15      51
L65      51
M85      51
K21      44
D69      42
N18      40
D35      40
Z36      39
D63      39
N72      39
R60      37
O99      37
O36      37
C61      36
R41      34
R87      34
R00      34
E66      34
J01      34
E87      34
M54      33
M48      33
R79      33
R76      33
Z86      32
C20      29
J45      29
R25      28
E51      28
H53      28
N28      28
O34      28
H26      28
F51      28
Q51      27
M19      27
C50      27
R30      26
N03      26
E28      25
N90      25
Z87      23
O26      22
R68 

In [72]:
#dataset looks good and there no puntuactions in column code
# join icd_breakdown dataset with valid_icd_10_codes to generate claims with valid code
valid_icd = read_file('valid_icd_10_codes.csv')
#valid_icd.head()
#valid_icd.info()
valid_diagnosis_df = icd_breakdown.merge(valid_icd, on='code', how='inner')
valid_diagnosis_df.head()

,claim_id,patient_id,diagnosis_codes,procedure_code,date_service,date_received,code
0,1.0,A1670,Z01.419,99999,1/25/2021,1/26/2021,Z01
1,2.0,A0086,Z01.419,99999,1/27/2021,1/29/2021,Z01
2,8.0,A0311,Z01.419,83615,1/14/2021,1/15/2021,Z01
3,12.0,A0311,Z01.419,99999,1/28/2021,1/30/2021,Z01
4,17.0,A0311,Z01.818,87389,1/21/2021,1/22/2021,Z01


Top 5 most common valid diagnosis code

In [73]:
valid_diagnosis_df['code'].value_counts(ascending=False).head()

Z11    1474
Z34     861
Z01     529
E55     498
Z12     427
Name: code, dtype: int64

The frequency table above shows the <b>top most 5 valid diagnosis codes.

### Q3. Data Quality Check


* From the table definition , patient_id, claim_id, procedure code, date_services, and date_received of sample_claims
  should not have null values
* For table valid_icd_10_codes, code column should not have null values
* For table valid_cpt_codes should not have null values

In [74]:
# let check if there are null values for the columns that are supposed to not have null values
claims.isnull().sum()

claim_id              2
patient_id            6
diagnosis_codes    1286
procedure_code        3
date_service         10
date_received         0
dtype: int64

From the above analysis, we can see that claim_id, patient_id, procedure_code, and date_service columns have 2, 6, 3, and 10 null values. According to the table definition there shouldn't be null values for these columns. It will cause issues when loading the data into database. 

In [75]:
valid_procedure.isnull().sum()

code                 0
short_description    0
procedure_code       0
dtype: int64

In [76]:
valid_icd.isnull().sum()

code    0
dtype: int64

In [77]:
# lets check how many invalid codes data are there in the sample claims datasets

no_invalid_procedure_codes= (claims['procedure_code'].value_counts().sum()) -(valid_code_claim['procedure_code'].value_counts().sum())

print('# row in sample claims with invalid procedure code :', no_invalid_procedure_codes)
print('# of invalid diagnosis_code:' '440')


# row in sample claims with invalid procedure code : 1693
# of invalid diagnosis_code:440


In [82]:
def convert_date(file_name, col):
    file_name[col]=pd.to_datetime(file_name[col], errors='coerce')
    return file_name

new_df =convert_date(claims, 'date_service')
new_df=convert_date(claims, 'date_received')
new_df[new_df['date_service'].isna()]
    

,claim_id,patient_id,diagnosis_codes,procedure_code,date_service,date_received
570,572.0,A1415,M15.0^L65.9^M85.9^E55.9^R73.09^E78.5,86850,NaT,2021-01-29
720,724.0,A1517,Z01.419^N89.8^Z11.3,99999,NaT,2021-01-27
932,934.0,A1580,Z11.3^Z34.81,85049,NaT,2021-01-21
985,987.0,A1580,N76.0,99999,NaT,2021-01-09
1589,1595.0,A1617,Z00.00^Z11.3^Z11.59^Z13.228^Z13.29^Z13.79^Z13....,82247,NaT,2021-01-05
1613,1619.0,A1617,J45,84460,NaT,2021-01-13
1848,1855.0,A1617,J45,99999,NaT,2021-01-30
2591,2593.0,A1664,Z34.02,88175,NaT,2021-01-07
3375,3379.0,A1668,R87.610^N95.0,85049,NaT,2021-01-27
4260,4267.0,A3369,B18.2^R19.7,85014,NaT,2021-01-20


There are 20 rows with invaid date format including null values. The dataframe above identifies the rows with invalid dateformat and rows having null values in date_service column.

In [79]:
df1= new_df[new_df['date_service'].notna()]
df1.head()

,claim_id,patient_id,diagnosis_codes,procedure_code,date_service,date_received
0,1.0,A1670,Z01.419^Z11.51,99999,2021-01-25,1/26/2021
1,2.0,A0086,Z01.419^Z12.4,99999,2021-01-27,1/29/2021
2,3.0,A0086,Z12.4^Z11.51,87491,2021-01-07,1/10/2021
3,4.0,A0086,R30.0,86735,2021-01-15,1/16/2021
4,5.0,A0311,Z34.81^I10^G40.909^E66.9,83014,2021-01-06,1/7/2021


In [80]:
def test_date(df, year, month, col):     
    df_no_null = df[df[col].notnull()]     
    return df_no_null[(df_no_null[col].dt.year != year) | (df_no_null[col].dt.month != month) ]
test_date(new_df, 2021, 1, 'date_service')

,claim_id,patient_id,diagnosis_codes,procedure_code,date_service,date_received
125,125.0,A0482,Z34.82^O09.32,N92.6,2141-05-30,1/1/2021
593,595.0,A1415,Z31.83^Z11.3^Z31.49^Z13.29^Z11.59^Z11.4^Z00.00...,Z13.79,2173-10-13,1/18/2021
765,768.0,A1549,Z34.01^O34.01,Q51.4,2173-10-13,1/1/2021
995,997.0,A1580,M15.0^L65.9^M85.9^E55.9^R73.09,E78.5,2130-05-26,1/7/2021
1523,1528.0,A1617,NaN,87389,2022-01-31,2/1/2021
1645,1651.0,A1617,NaN,84155,2022-01-31,2/1/2021
1651,1657.0,A1617,NaN,Z01.419,2137-07-17,1/5/2021
1802,1809.0,A1617,Z83.3^E55.9,R63.5,2133-07-22,1/12/2021
1903,1909.0,A1623,Z01.419^Z11.3,Z11.8,2173-10-13,1/27/2021
1963,1969.0,A1623,F41.9^R00.2^R73.03,G47.9,2124-08-12,1/16/2021


In [83]:
new_df.dtypes

claim_id                  float64
patient_id                 object
diagnosis_codes            object
procedure_code             object
date_service       datetime64[ns]
date_received      datetime64[ns]
dtype: object

In [84]:
def test_date(df, year, col):     
    df_no_null = df[df[col].notnull()]     
    return df_no_null[(df_no_null[col].dt.year < 2021)]
test_date(new_df, 2021, 'date_received')

,claim_id,patient_id,diagnosis_codes,procedure_code,date_service,date_received
50,51.0,A0311,N76.0,86850,2021-01-05,2020-12-26
491,494.0,A1406,C50.011,99999,2021-01-07,2020-12-28
1167,1172.0,A1616,E78.5^M25.50^Z79.899^Z12.5^D35.2,84478,2021-01-04,2020-12-25
1467,1472.0,A1617,Z31.83^Z11.3^Z31.49^Z13.29^Z11.59^Z11.4^Z00.00...,83021,2021-01-06,2020-12-27
1541,1546.0,A1617,Z34.81^Z20.2^F51.4^Z86.718,99999,2021-01-06,2020-12-27
2405,2410.0,A1623,Z34.82^O36.0990,83520,2021-01-08,2020-12-29
2517,2520.0,A1655,Z34.81,87798,2021-01-06,2020-12-27
3141,3144.0,A1665,NaN,85049,2021-01-04,2020-12-25
3696,3705.0,A1696,Z83.3^E55.9^R63.5,99999,2021-01-06,2020-12-27
3831,3839.0,A3185,R05^I10^E03.8^Z79.899,87799,2021-01-07,2020-12-28


There are 13 rows where date_received are in 2020 but date_service are in 2021. The date_received can't be less than date_service. 

In [57]:
def service_date_greater(df, col1, col2):
    x= df[col1]>new_df[col2]
    return df[x]

date_not_chro= service_date_greater(new_df, 'date_service', 'date_received')
date_not_chro.shape


(81, 6)

There are 81 rows where date_service greater than date_received, which need further investigation if date_service has to be less than the date_received.